# Import Needed Libraries and Set Options

In [24]:
import pandas as pd
import numpy as np

print('Libraries imported')

Libraries imported


In [25]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print('Option set to display all dataframe rows')

Option set to display all dataframe rows


#### 

# Scrape Table from Wikipedia Page

In [32]:
postals = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0] #only want the first table in the wikipedia page
postals = postals[postals['Borough'] != 'Not assigned'].reset_index().drop('index', axis = 1)
postals.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [27]:
postals.shape

(103, 3)

#### 

# Obtain Geographical Coordinates for Each Neighbourhood and Add to Dataframe

(I could not get geocoder to work, so I used the provided csv file)

In [28]:
coords = pd.read_csv("http://cocl.us/Geospatial_data")
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Get shape of coord dataframe (to ensure as expected)

In [29]:
coords.shape

(103, 3)

Add coordiantes to postals dataframe

In [33]:
postals = postals.merge(coords, on = 'Postal Code').sort_values(by = 'Postal Code')
postals.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
22,M1G,Scarborough,Woburn,43.770992,-79.216917
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### 

# Cluster Neighbourhoods in Toronto
(I will cluster neighborhoods based on number of shared venue types using the KMeans algorithm)

### Import Libraries Needed for Clustering and Visualization

In [21]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# import k-means from clustering stage
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print("Libraries imported")

Libraries imported


### Get Coordinates of Toronto

In [22]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

tor_coords = [latitude,longitude]

print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


### Create Map of Toronto with Postal Code Markers Overlapped


In [38]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=tor_coords, zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(postals['Latitude'], postals['Longitude'], postals['Borough'], postals['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [35]:
CLIENT_ID = 'C5TVLNNOZ3QX14URLNVGVXAKYHOT421V0KR21MSGVJY1O2CR' # your Foursquare ID
CLIENT_SECRET = 'BCTZ4WTEUPUZWKQO0RBRZQZUJ3QDSXV1Y2BKIW5VGUP32M2H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C5TVLNNOZ3QX14URLNVGVXAKYHOT421V0KR21MSGVJY1O2CR
CLIENT_SECRET: BCTZ4WTEUPUZWKQO0RBRZQZUJ3QDSXV1Y2BKIW5VGUP32M2H


### Get Venues in Each Toronto Neighborhood in a Dataframe

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', r
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### ### ###

tor_neighbs_vens = getNearbyVenues(names=postals['Neighbourhood'],
                                 latitudes=postals['Latitude'],
                                 longitudes=postals['Longitude']
                                )

tor_neighbs_vens.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Get shape of venues dataframe

In [41]:
tor_neighbs_vens.shape

(2146, 7)

### Create Dataframe with Number of Venues in Each Neighborhood (Just for Reference)

In [43]:
tor_neighbs_vens_counts = tor_neighbs_vens.groupby(by = 'Neighbourhood').count().reset_index().loc[:, ['Neighbourhood','Venue']]
tor_neighbs_vens_counts.columns = ['Neighbourhood', 'Number of Venues']

tor_neighbs_vens_counts

,Neighbourhood,Number of Venues
0,Agincourt,5
1,"Alderwood, Long Branch",9
2,"Bathurst Manor, Wilson Heights, Downsview North",21
3,Bayview Village,4
4,"Bedford Park, Lawrence Manor East",26
5,Berczy Park,58
6,"Birch Cliff, Cliffside West",4
7,"Brockton, Parkdale Village, Exhibition Place",22
8,"Business reply mail Processing Centre, South C...",14
9,"CN Tower, King and Spadina, Railway Lands, Har...",17


In [44]:
tor_neighbs_vens_counts.shape

(95, 2)

### Code Neighborhood Venues for Clustering

In [46]:
tor_neighbs_vens_onehot = pd.get_dummies(tor_neighbs_vens[['Venue Category']], prefix="", prefix_sep="")

tor_neighbs_vens_onehot['Neighbourhood'] = tor_neighbs_vens['Neighbourhood']

columns = tor_neighbs_vens_onehot.columns.tolist()
columns.remove('Neighbourhood')
reordered_columns = ['Neighbourhood'] + columns

tor_neighbs_vens_onehot = tor_neighbs_vens_onehot[reordered_columns]

tor_neighbs_vens_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Find the Number of Most-Prevalent Venue Type in Each Neighborhood

In [47]:
tor_neighbs_vens_onehot_grouped = tor_neighbs_vens_onehot.groupby(by = 'Neighbourhood').sum().reset_index()

#check to see if any neighborhoods have more than one instance of a venue type (to see if one_hot coding worked
#I'd expect more than one instance of at least one type of venue in each neighborhood))
for index, row in tor_neighbs_vens_onehot_grouped.iterrows():
    print(row[0], ':', max(row[1:]))

Agincourt : 1
Alderwood, Long Branch : 2
Bathurst Manor, Wilson Heights, Downsview North : 2
Bayview Village : 1
Bedford Park, Lawrence Manor East : 2
Berczy Park : 4
Birch Cliff, Cliffside West : 1
Brockton, Parkdale Village, Exhibition Place : 3
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto : 1
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport : 3
Caledonia-Fairbanks : 2
Canada Post Gateway Processing Centre : 2
Cedarbrae : 1
Central Bay Street : 11
Christie : 4
Church and Wellesley : 7
Clarks Corners, Tam O'Shanter, Sullivan : 2
Cliffside, Cliffcrest, Scarborough Village West : 1
Commerce Court, Victoria Hotel : 12
Davisville : 3
Davisville North : 1
Del Ray, Mount Dennis, Keelsdale and Silverthorn : 1
Don Mills : 3
Dorset Park, Wexford Heights, Scarborough Town Centre : 2
Downsview : 3
Dufferin, Dovercourt Village : 2
East Toronto, Broadview North (Old East York) : 1
Eringate, Bloordale 

In [48]:
tor_neighbs_vens_onehot_grouped.shape

(95, 270)

### Cluster the Neighborhoods According to Venue Type Counts

Create dataframe to record cluster of each neighborhood

In [49]:
postals_clusted = pd.DataFrame(columns = ['Neighbourhood', 'Cluster'])

neighbs = tor_neighbs_vens_onehot_grouped['Neighbourhood'].tolist()
postals_clusted['Neighbourhood'] = neighbs

postals_clusted = postals_clusted.merge(postals[['Neighbourhood','Borough']], on = 'Neighbourhood').drop_duplicates()
postals_clusted = postals_clusted[['Neighbourhood','Borough','Cluster']]

postals_clusted['Cluster'] = ['Not Assigned'] * len(neighbs)

postals_clusted.head(10)

,Neighbourhood,Borough,Cluster
0,Agincourt,Scarborough,Not Assigned
1,"Alderwood, Long Branch",Etobicoke,Not Assigned
2,"Bathurst Manor, Wilson Heights, Downsview North",North York,Not Assigned
3,Bayview Village,North York,Not Assigned
4,"Bedford Park, Lawrence Manor East",North York,Not Assigned
5,Berczy Park,Downtown Toronto,Not Assigned
6,"Birch Cliff, Cliffside West",Scarborough,Not Assigned
7,"Brockton, Parkdale Village, Exhibition Place",West Toronto,Not Assigned
8,"Business reply mail Processing Centre, South C...",East Toronto,Not Assigned
9,"CN Tower, King and Spadina, Railway Lands, Har...",Downtown Toronto,Not Assigned


Get shape of above dataframe

In [50]:
postals_clusted.shape

(95, 3)

Fit clustering algorithm and report labels

In [52]:
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_neighbs_vens_onehot_grouped.drop("Neighbourhood", axis = 1))

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 1, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 3, 1, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1,
       0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 1, 2, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0], dtype=int32)

Get length of cluster label list (to ensure is as expected)

In [53]:
len(kmeans.labels_)
#this means 94 neighborhoods had venues

95

### Add Assigned Cluster to Original 'Postals' Dataframe

In [54]:
#get list of neighborhoods that have venues
postals_clusted['Cluster'] = kmeans.labels_
postals_clusted.head()

,Neighbourhood,Borough,Cluster
0,Agincourt,Scarborough,0
1,"Alderwood, Long Branch",Etobicoke,0
2,"Bathurst Manor, Wilson Heights, Downsview North",North York,0
3,Bayview Village,North York,0
4,"Bedford Park, Lawrence Manor East",North York,0


### Add Coordinates to Dataframe Showing Neighborhood Cluster Assignment

In [55]:
lat_long_dict = {}

for i,row in postals.iterrows():
    lat_long_dict[row["Neighbourhood"]] = [row['Latitude'],row['Longitude']]

#### #### #### ####

lats = []

for n in postals_clusted['Neighbourhood']:
    lats.append(lat_long_dict[n][0])

postals_clusted['Latitude'] = lats
postals_clusted.head()

#### #### #### ####

longs = []

for n in postals_clusted['Neighbourhood']:
    longs.append(lat_long_dict[n][1])

postals_clusted['Longitude'] = longs
postals_clusted.head()

,Neighbourhood,Borough,Cluster,Latitude,Longitude
0,Agincourt,Scarborough,0,43.794200,-79.262029
1,"Alderwood, Long Branch",Etobicoke,0,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",North York,0,43.754328,-79.442259
3,Bayview Village,North York,0,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",North York,0,43.733283,-79.419750


#### 

# Final Clustering Report

### Number of Neighborhoods Assigned to Each Cluster

In [56]:
postals_clusted.groupby(by = 'Cluster').count()

,Neighbourhood,Borough,Latitude,Longitude
Cluster,,,,
0,73,73,73,73
1,6,6,6,6
2,14,14,14,14
3,2,2,2,2


### List of Neighborhoods in Cluster 0

In [57]:
postals_clusted[postals_clusted['Cluster'] == 0]

,Neighbourhood,Borough,Cluster,Latitude,Longitude
0,Agincourt,Scarborough,0,43.794200,-79.262029
1,"Alderwood, Long Branch",Etobicoke,0,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",North York,0,43.754328,-79.442259
3,Bayview Village,North York,0,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",North York,0,43.733283,-79.419750
6,"Birch Cliff, Cliffside West",Scarborough,0,43.692657,-79.264848
7,"Brockton, Parkdale Village, Exhibition Place",West Toronto,0,43.636847,-79.428191
8,"Business reply mail Processing Centre, South C...",East Toronto,0,43.662744,-79.321558
9,"CN Tower, King and Spadina, Railway Lands, Har...",Downtown Toronto,0,43.628947,-79.394420
10,Caledonia-Fairbanks,York,0,43.689026,-79.453512


### List of Neighborhoods in Cluster 1

In [58]:
postals_clusted[postals_clusted['Cluster'] == 1]

,Neighbourhood,Borough,Cluster,Latitude,Longitude
18,"Commerce Court, Victoria Hotel",Downtown Toronto,1,43.648198,-79.379817
33,"First Canadian Place, Underground city",Downtown Toronto,1,43.648429,-79.382280
39,"Harbourfront East, Union Station, Toronto Islands",Downtown Toronto,1,43.640816,-79.381752
69,"Richmond, Adelaide, King",Downtown Toronto,1,43.650571,-79.384568
80,Stn A PO Boxes,Downtown Toronto,1,43.646435,-79.374846
88,"Toronto Dominion Centre, Design Exchange",Downtown Toronto,1,43.647177,-79.381576


### List of Neighborhoods in Cluster 2

In [59]:
postals_clusted[postals_clusted['Cluster'] == 2]

,Neighbourhood,Borough,Cluster,Latitude,Longitude
5,Berczy Park,Downtown Toronto,2,43.644771,-79.373306
13,Central Bay Street,Downtown Toronto,2,43.657952,-79.387383
15,Church and Wellesley,Downtown Toronto,2,43.665860,-79.383160
19,Davisville,Central Toronto,2,43.704324,-79.388790
48,"Kensington Market, Chinatown, Grange Park",Downtown Toronto,2,43.653206,-79.400049
53,"Little Portugal, Trinity",West Toronto,2,43.647927,-79.419750
67,"Queen's Park, Ontario Provincial Government",Downtown Toronto,2,43.662301,-79.389494
68,"Regent Park, Harbourfront",Downtown Toronto,2,43.654260,-79.360636
73,"Runnymede, Swansea",West Toronto,2,43.651571,-79.484450
77,St. James Town,Downtown Toronto,2,43.651494,-79.375418


### List of Neighborhoods in Cluster 3

In [60]:
postals_clusted[postals_clusted['Cluster'] == 3]

,Neighbourhood,Borough,Cluster,Latitude,Longitude
32,"Fairview, Henry Farm, Oriole",North York,3,43.778517,-79.346556
35,"Garden District, Ryerson",Downtown Toronto,3,43.657162,-79.378937


### Map the Neighborhoods Color-Coded by Cluster

In [61]:
map_clusters = folium.Map(location=tor_coords, zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors = ['Red', 'Green', 'Blue', 'Purple']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(postals_clusted['Latitude'], postals_clusted['Longitude'], postals_clusted['Neighbourhood'], postals_clusted['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)

print('\n')       
for i in range(4):
    print('Cluster ', str(i), '= ', colors[i])
print('\n') 

map_clusters



Cluster  0 =  Red
Cluster  1 =  Green
Cluster  2 =  Blue
Cluster  3 =  Purple


